In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import math 
import sys

In [5]:
movie_budgets = pd.read_csv('movie_budgets_clean.csv')
rt_movie = pd.read_csv('rt_movie_clean.csv')